In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import numpy as np
from nltk.corpus import brown
from  gensim.utils import simple_preprocess
from tqdm import tqdm
import random



In [2]:
brown_words = brown.words()
brown_text = " ".join(brown.words()).lower()

In [3]:
num_sents = len(brown.sents())
print("number of sentences:", num_sents)

number of sentences: 57340


In [4]:
len(np.unique(simple_preprocess(brown_text.lower())))

41239

In [5]:
a = {"x":4, "y":3}
list(a.keys())

['x', 'y']

In [5]:
class Word2VecModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2VecModel, self).__init__()
        self.target_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)
        self.context_embeddings = nn.Embedding(vocab_size, embedding_dim, sparse=True)

        # Initialize weights
        self.target_embeddings.weight.data.uniform_(-1, 1)
        self.context_embeddings.weight.data.uniform_(-1, 1)

    def forward(self, target_words, context_words, negative_words):
        target_embeds = self.target_embeddings(target_words)
        context_embeds = self.context_embeddings(context_words)
        negative_embeds = self.context_embeddings(negative_words)

        positive_score = torch.sum(target_embeds * context_embeds, dim=1)
        negative_score = torch.bmm(negative_embeds, target_embeds.unsqueeze(2)).squeeze()

        return -torch.mean(torch.log(torch.sigmoid(positive_score)) + torch.sum(torch.log(torch.sigmoid(-negative_score)), dim=1))


In [6]:
def preprocess_brown_corpus():
    processed_sentences = []
    for sentence in brown.sents():
        processed_sentence = simple_preprocess(' '.join(sentence), deacc=True)  
        processed_sentences.append(processed_sentence)
    return processed_sentences

In [10]:
def prepare_dataset(batch_size=512, window_size=5, negative_samples=5, total_negative_samples=10000):
    # Example corpus
    corpus = preprocess_brown_corpus()
    print("preprocessed")
    # Hyperparameters
    

    # Prepare dataset and dataloader
    dataset = Word2VecSentenceDataset(corpus, window_size, negative_samples, total_negative_samples)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    print("model loaded")
    return dataset, dataloader


In [8]:

def train(dataset, dataloader, epochs = 10, embedding_dim = 20):

     
     learning_rate = 0.01

     vocab_size = len(dataset.vocab)
     model = Word2VecModel(vocab_size, embedding_dim)
     
     optimizer = optim.SparseAdam(model.parameters(), lr=learning_rate)



     # Example training loop
     for epoch in range(epochs):
          total_loss = 0
          for target, context, negatives in tqdm(dataloader):  
               model.zero_grad()
               loss = model(target, context, negatives)
               loss.backward()
               optimizer.step()
               total_loss += loss.item()
               
          
          print(f"Epoch {epoch}, Loss: {total_loss}")

     return model


In [11]:
from dataset import Word2VecSentenceDataset

dataset, dataloader = prepare_dataset(batch_size=512, window_size=5, negative_samples=5, total_negative_samples=10000)
model = train(dataset, dataloader, epochs=5, embedding_dim = 30)

preprocessed
model loaded


100%|██████████| 15925/15925 [03:13<00:00, 82.19it/s]


Epoch 0, Loss: 8205.604247227311


100%|██████████| 15925/15925 [03:16<00:00, 81.22it/s]


Epoch 1, Loss: 4845.350632846355


...

Epoch 2, Loss: 4641.762401416898


100%|██████████| 15925/15925 [04:47<00:00, 55.47it/s]


Epoch 3, Loss: 4491.427864342928


100%|██████████| 15925/15925 [04:14<00:00, 62.49it/s]

Epoch 4, Loss: 4375.1427896767855


In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], profile_memory=True) as prof:
    with record_function("model_inference"):
        model = train(dataset, dataloader)
print("xxxxxxxxxx")
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


In [ ]:
import cProfile
import pstats

# Create a Profile object
profiler = cProfile.Profile()
profiler.enable()

# The code you want to profile
prepare_dataset()

profiler.disable()

# Create Stats object
stats = pstats.Stats(profiler).sort_stats('time')

# Print the statistics
stats.print_stats()




In [14]:
prof.key_averages().table(sort_by="cpu_time_total", row_limit=10)

''

In [12]:
# Inspect embeddings
word_embeddings = model.target_embeddings.weight.data
print(word_embeddings[dataset.vocab["atlanta"]])

tensor([ 0.8798, -0.0981, -0.0641, -0.6053, -0.0116,  0.1742,  0.4074, -0.1351,
        -0.3801,  0.5048, -1.6327,  0.0111, -0.0943, -0.1630,  1.1513,  0.1729,
        -0.1996, -0.7478, -1.0341,  0.5263,  0.4483,  0.1727,  1.0539, -0.1429,
        -0.2299,  0.7460,  0.1045,  0.5899, -0.3256, -0.1430])


In [13]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def find_most_similar(word, word_to_int, int_to_word, embeddings, top_n=5):
    # Get the embedding for the given word
    word_idx = word_to_int[word]
    word_embedding = embeddings[word_idx].reshape(1, -1)
    
    # Calculate cosine similarity between this word and all other words in the vocabulary
    similarities = []
    for i in range(len(embeddings)):
        other_word_embedding = embeddings[i].reshape(1, -1)
        similarity = cosine_similarity(word_embedding, other_word_embedding)[0][0]
        similarities.append((i, similarity))
    
    # Sort by similarity
    similarities.sort(key=lambda x: x[1], reverse=True)
    
    # Convert indices back to words and filter out the input word
    similar_words = [(int_to_word[sim[0]], sim[1]) for sim in similarities if sim[0] != word_idx]
    
    # Return the top N most similar words, excluding the word itself
    return similar_words[:top_n]




In [14]:
# Example usage
word = 'book'  # The word you want to find similar words for
similar_words = find_most_similar(word, dataset.vocab, dataset.index_to_word, word_embeddings, top_n=5)
print(f"Most similar words to '{word}':", similar_words)

Most similar words to 'book': [('comedie', 0.8072341), ('trempler', 0.8050022), ('often', 0.80381423), ('expe', 0.8015127), ('hors', 0.80094683)]


In [15]:
# Example usage
word = 'twenty'  # The word you want to find similar words for
similar_words = find_most_similar(word, dataset.vocab, dataset.index_to_word, word_embeddings, top_n=5)
print(f"Most similar words to '{word}':", similar_words)

Most similar words to 'twenty': [('thirty', 0.8679474), ('ten', 0.8436272), ('six', 0.8391596), ('laid', 0.8352573), ('five', 0.82961893)]
